In [2]:
import pandas as pd
import numpy as np
import json
import datetime as dt

In [3]:
data = pd.read_csv('main_task.csv')

In [4]:
data.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [6]:
for el in data.iloc[0]:
    print(type(el), el)

<class 'str'> id_5569
<class 'str'> Paris
<class 'str'> ['European', 'French', 'International']
<class 'numpy.float64'> 5570.0
<class 'numpy.float64'> 3.5
<class 'str'> $$ - $$$
<class 'numpy.float64'> 194.0
<class 'str'> [['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]
<class 'str'> /Restaurant_Review-g187147-d1912643-Reviews-R_Yves-Paris_Ile_de_France.html
<class 'str'> d1912643


In [7]:
data['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [8]:
len(data.City.value_counts())

31

In [9]:
def get_cuisines(cuisines):
    '''
    Получаем список кухонь в виде:
    cuisines[0] - list of cusines
    если был NaN, то возвращается 'Regionl Cusine' -как самая популярная в регионе/городе/стране
    '''
    if cuisines == 'NaN': return ['Regionl Cusine']#['Vegetarian Friendly'] # 'Vegetarian Friendly' - самая популярная
    if  cuisines:
        cuisines = str(cuisines).replace("'",'"')
        return json.loads(cuisines)
    else:
        return ['Regionl Cusine'] #return ['Vegetarian Friendly']

In [10]:
allCusines = []
def cuisine_styles_count(row):
    '''
    Получаем количество кухонь
    '''
    global allCusines
    cusines = get_cuisines(row['Cuisine Style'])
    
    if row['Cuisine Style'] != 'NaN':    
        cusines = get_cuisines(row['Cuisine Style'])
        allCusines.extend(cusines)
        cuisines_count =len(cusines)
    else:
        cuisines_count = 1

    return cuisines_count

In [11]:
df = data
df['Cuisine Style NAN'] = df['Cuisine Style'].isna()
df['Cuisine Style'] = df['Cuisine Style'].fillna('NaN')
df['Cuisines Count'] = df.apply(cuisine_styles_count, axis=1)

most_popular_cusine = pd.Series(allCusines).value_counts().index[0]
average_cousines_count = np.round(df['Cuisines Count'].mean(),1)

In [12]:
most_popular_cusine

'Vegetarian Friendly'

In [13]:
average_cousines_count

2.6

In [14]:
len(set(allCusines))

125

In [37]:
CURRENT_DATE = pd.to_datetime('29/10/2020')

In [34]:
# вытащим дату из ревью и создадим новые критерии
df['date_of_Review'] = df['Reviews'].str.findall('\d+/\d+/\d+')
df['len_date'] = df['date_of_Review'].apply(lambda x: len(x))

print("кол-во значений Reviews с тремя датами :=" , len(df[df['len_date']==3]))

кол-во значений Reviews с тремя датами := 2


In [36]:
df[df['len_date']==3]

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Cuisine Style NAN,Cuisines Count,date_of_Review,len_date
9254,id_12965,London,"['Cafe', 'French']",12975.0,3.5,$$ - $$$,31.0,"[['The desserts arcade !', 'Visit today 3/1/20...",/Restaurant_Review-g186338-d1017372-Reviews-Ma...,d1017372,False,2,"[3/1/2017, 06/02/2017, 01/03/2017]",3
23348,id_450,Geneva,"['French', 'European']",453.0,4.0,$$ - $$$,48.0,"[['lunch 29/9/2016', 'Unplanned Great Dinner']...",/Restaurant_Review-g188057-d2706773-Reviews-Le...,d2706773,False,2,"[29/9/2016, 09/30/2016, 08/25/2016]",3


In [38]:
print("значения Reviews с тремя датами :=")
temp_list = df[df['len_date']==3].Reviews.to_list()
display(df[df['len_date']==3].Reviews.to_list())

значения Reviews с тремя датами :=


["[['The desserts arcade !', 'Visit today 3/1/2017'], ['06/02/2017', '01/03/2017']]",
 "[['lunch 29/9/2016', 'Unplanned Great Dinner'], ['09/30/2016', '08/25/2016']]"]

In [39]:
df['len_date'].date_of_Review = df[df['len_date']==3].date_of_Review.apply(lambda x: x.pop(0))

In [42]:
print("кол-во значений Reviews с одной датой :=" , len(df[df['len_date']==1]))
display(df[df['len_date']==1].Reviews[:4])

кол-во значений Reviews с одной датой := 4556


16    [['This is a real hidden Sushi-gem'], ['04/12/...
26    [['Heavenly meat slices and craft beer too.'],...
27    [['Good value canteen lunch stop.'], ['01/26/2...
28                       [['Fab Food'], ['08/20/2011']]
Name: Reviews, dtype: object

In [46]:
df['one_Review'] = (df['len_date']==1).astype('float64')

# заполним перерыв между отзывами (по отзывам где len = 2) и насколько давно был сделан последний самый свежий отзыв
# создадим для этого функции:
def time_to_now(row):
    if row['date_of_Review'] == []:
        return None
    return dt.datetime.now() - pd.to_datetime(row['date_of_Review']).max()

def time_between_Reviews(row):
    if row['date_of_Review'] == []:
        return None
    return pd.to_datetime(row['date_of_Review']).max() - pd.to_datetime(row['date_of_Review']).min()

df['day_to_now'] = df.apply(time_to_now, axis = 1).dt.days
df['day_between_Reviews'] = df[df['len_date']==2].apply(time_between_Reviews, axis = 1).dt.days

In [47]:
df['day_between_Reviews'].max()

3207.0

In [49]:
df['date_of_Review'][df['day_to_now']==df['day_to_now'].min()]

1592     [02/26/2018, 07/24/2017]
4000     [02/26/2018, 02/15/2018]
5635     [02/26/2018, 01/02/2018]
7866     [02/26/2018, 02/24/2018]
8135     [02/26/2018, 02/04/2018]
10600    [02/26/2018, 01/31/2018]
12019    [02/26/2018, 01/11/2018]
15088    [02/26/2018, 02/14/2018]
17447    [02/26/2018, 02/09/2017]
18511    [02/26/2018, 02/25/2018]
19050    [02/26/2018, 02/15/2018]
22263    [02/26/2018, 02/24/2018]
22880    [02/26/2018, 02/24/2018]
24797    [02/26/2018, 01/24/2018]
25110    [02/26/2018, 02/24/2018]
25943    [02/26/2018, 02/26/2018]
26781    [02/26/2018, 02/22/2018]
28928    [02/26/2018, 01/14/2018]
29251    [02/26/2018, 02/23/2018]
30334    [02/26/2018, 02/24/2018]
33662    [02/26/2018, 05/01/2017]
37663    [02/26/2018, 01/21/2018]
37689    [02/26/2018, 05/31/2017]
38749    [02/26/2018, 01/30/2017]
Name: date_of_Review, dtype: object

In [50]:
dic_value_Price = {'$':1,'$$ - $$$':2,'$$$$':3}
df['Price_Range']=df['Price Range'].map(lambda x: dic_value_Price.get(x,x))

# 18412 ресторанов это более 70% из заполненной информации имеют средний параметр цены
# поэтому заполняем пропуски двойкой (2)
df['Price_Range'] = df['Price_Range'].fillna(2)

In [51]:
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,Cuisine Style NAN,Cuisines Count,date_of_Review,len_date,one_Review,day_to_now,day_between_Reviews,Price_Range
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,False,3,"[12/31/2017, 11/20/2017]",2,0.0,1033.0,41.0,2.0
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,True,1,"[07/06/2017, 06/19/2016]",2,0.0,1211.0,382.0,2.0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,False,7,"[01/08/2018, 01/06/2018]",2,0.0,1025.0,2.0,3.0
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,True,1,[],0,0.0,NaN,NaN,2.0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,False,3,"[11/18/2017, 02/19/2017]",2,0.0,1076.0,272.0,2.0


In [57]:
X = df[['Price_Range', 'Ranking', 'day_between_Reviews', 'one_Review',  'Cuisines Count','Number of Reviews']].fillna(0)
y = df['Rating']
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.3874485873015873


In [58]:
df.drop(['Restaurant_id','URL_TA', 'ID_TA'], axis = 1, inplace=True)

In [59]:
df['isMostPopCusine'] = df['Cuisine Style'].apply(lambda x: 1 if most_popular_cusine in x else 0 )
df['isMultyCusine'] = df['Cuisines Count'].apply(lambda x: 1 if  x >= average_cousines_count else 0 )

In [60]:
df

,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,Cuisine Style NAN,Cuisines Count,date_of_Review,len_date,one_Review,day_to_now,day_between_Reviews,Price_Range,isMostPopCusine,isMultyCusine
0,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",False,3,"[12/31/2017, 11/20/2017]",2,0.0,1033.0,41.0,2.0,0,1
1,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",True,1,"[07/06/2017, 06/19/2016]",2,0.0,1211.0,382.0,2.0,0,0
2,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",False,7,"[01/08/2018, 01/06/2018]",2,0.0,1025.0,2.0,3.0,1,1
3,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",True,1,[],0,0.0,NaN,NaN,2.0,0,0
4,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",False,3,"[11/18/2017, 02/19/2017]",2,0.0,1076.0,272.0,2.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",False,4,"[12/16/2017, 11/12/2017]",2,0.0,1048.0,34.0,2.0,1,1
39996,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",False,5,"[12/21/2017, 12/12/2017]",2,0.0,1043.0,9.0,2.0,1,1
39997,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",False,2,"[11/03/2016, 04/12/2008]",2,0.0,1456.0,3127.0,2.0,0,0
39998,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",False,5,"[07/11/2017, 06/18/2017]",2,0.0,1206.0,23.0,2.0,1,1


In [61]:
df_City_dummies = pd.get_dummies(df['City'], dummy_na=False).astype('float64')
df = pd.concat([df,df_City_dummies], axis=1)

In [66]:
X = df.drop(['City', 'Cuisine Style', 'Price Range', 'Reviews', 'Cuisine Style NAN', 'date_of_Review','Rating'], axis = 1).fillna(0)
y = df['Rating']

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21347149999999998
